# Practical FFT Applications

This notebook demonstrates real-world applications of FFT including spectral analysis, noise filtering, and windowing.

In [1]:
import numpy as np
import matplotlib.pyplot as plt

try:
    from ipywidgets import interact, FloatSlider, IntSlider, Dropdown, Checkbox
    WIDGETS_AVAILABLE = True
except ImportError:
    WIDGETS_AVAILABLE = False
    print("ipywidgets not available. Install with: pip install ipywidgets")

%matplotlib inline
plt.rcParams['figure.figsize'] = (10, 4)

## 1. Interactive Low-Pass Filter

Remove high-frequency noise by filtering in the frequency domain.

In [2]:
def plot_lowpass_filter(signal_freq=10, noise_level=0.5, cutoff_freq=30):
    """Demonstrate low-pass filtering."""
    sample_rate = 1000
    duration = 2.0
    t = np.linspace(0, duration, int(sample_rate * duration), endpoint=False)
    
    # Create clean signal and add noise
    clean_signal = np.sin(2 * np.pi * signal_freq * t)
    np.random.seed(42)
    noise = noise_level * np.random.randn(len(t))
    noisy_signal = clean_signal + noise
    
    # FFT and filter
    n_samples = len(noisy_signal)
    fft_result = np.fft.rfft(noisy_signal)
    frequencies = np.fft.rfftfreq(n_samples, d=1/sample_rate)
    
    # Apply low-pass filter
    fft_filtered = fft_result.copy()
    fft_filtered[frequencies > cutoff_freq] = 0
    filtered_signal = np.fft.irfft(fft_filtered)
    
    # Plot
    fig, axes = plt.subplots(2, 2, figsize=(12, 8))
    
    axes[0, 0].plot(t, noisy_signal, 'b-', alpha=0.7)
    axes[0, 0].set_title(f'Noisy Signal ({signal_freq} Hz + noise)')
    axes[0, 0].set_ylabel('Amplitude')
    axes[0, 0].grid(True)
    
    axes[0, 1].plot(frequencies, np.abs(fft_result) / n_samples * 2, 'b-')
    axes[0, 1].axvline(x=cutoff_freq, color='r', linestyle='--', label=f'Cutoff: {cutoff_freq} Hz')
    axes[0, 1].axvspan(0, cutoff_freq, alpha=0.2, color='green', label='Pass band')
    axes[0, 1].set_title('Spectrum (Before Filtering)')
    axes[0, 1].set_ylabel('Magnitude')
    axes[0, 1].set_xlim(0, 100)
    axes[0, 1].legend()
    axes[0, 1].grid(True)
    
    axes[1, 0].plot(t, filtered_signal, 'g-', label='Filtered')
    axes[1, 0].plot(t, clean_signal, 'r--', alpha=0.5, label='Original')
    axes[1, 0].set_title('Filtered Signal')
    axes[1, 0].set_xlabel('Time (s)')
    axes[1, 0].set_ylabel('Amplitude')
    axes[1, 0].legend()
    axes[1, 0].grid(True)
    
    axes[1, 1].plot(frequencies, np.abs(fft_filtered) / n_samples * 2, 'g-')
    axes[1, 1].set_title('Spectrum (After Filtering)')
    axes[1, 1].set_xlabel('Frequency (Hz)')
    axes[1, 1].set_ylabel('Magnitude')
    axes[1, 1].set_xlim(0, 100)
    axes[1, 1].grid(True)
    
    plt.tight_layout()
    plt.show()
    
    error = np.sqrt(np.mean((clean_signal - filtered_signal)**2))
    print(f"RMS error vs original: {error:.4f}")

if WIDGETS_AVAILABLE:
    interact(
        plot_lowpass_filter,
        signal_freq=IntSlider(min=5, max=25, value=10, description='Signal (Hz)'),
        noise_level=FloatSlider(min=0, max=1.0, step=0.1, value=0.5, description='Noise Level'),
        cutoff_freq=IntSlider(min=10, max=80, value=30, description='Cutoff (Hz)')
    )
else:
    plot_lowpass_filter()

interactive(children=(IntSlider(value=10, description='Signal (Hz)', max=25, min=5), FloatSlider(value=0.5, de…

## 2. Interactive Band-Pass Filter

Extract only frequencies within a specific range.

In [3]:
def plot_bandpass_filter(f_low=3, f_mid=25, f_high=100, low_cutoff=15, high_cutoff=40):
    """Demonstrate band-pass filtering."""
    sample_rate = 1000
    duration = 2.0
    t = np.linspace(0, duration, int(sample_rate * duration), endpoint=False)
    
    # Create signal with three frequency components
    low_freq = np.sin(2 * np.pi * f_low * t)
    mid_freq = np.sin(2 * np.pi * f_mid * t)
    high_freq = np.sin(2 * np.pi * f_high * t)
    combined = low_freq + mid_freq + high_freq
    
    # FFT and band-pass filter
    n_samples = len(combined)
    fft_result = np.fft.rfft(combined)
    frequencies = np.fft.rfftfreq(n_samples, d=1/sample_rate)
    
    band_mask = (frequencies >= low_cutoff) & (frequencies <= high_cutoff)
    fft_bandpass = fft_result.copy()
    fft_bandpass[~band_mask] = 0
    extracted = np.fft.irfft(fft_bandpass)
    
    # Plot
    fig, axes = plt.subplots(2, 2, figsize=(12, 8))
    
    axes[0, 0].plot(t, combined, 'b-')
    axes[0, 0].set_title(f'Combined: {f_low} + {f_mid} + {f_high} Hz')
    axes[0, 0].set_ylabel('Amplitude')
    axes[0, 0].set_xlim(0, 0.5)
    axes[0, 0].grid(True)
    
    axes[0, 1].plot(frequencies, np.abs(fft_result) / n_samples * 2, 'b-')
    axes[0, 1].axvline(x=low_cutoff, color='r', linestyle='--')
    axes[0, 1].axvline(x=high_cutoff, color='r', linestyle='--')
    axes[0, 1].axvspan(low_cutoff, high_cutoff, alpha=0.2, color='green', label='Pass band')
    axes[0, 1].set_title(f'Spectrum (Band: {low_cutoff}-{high_cutoff} Hz)')
    axes[0, 1].set_ylabel('Magnitude')
    axes[0, 1].set_xlim(0, 150)
    axes[0, 1].legend()
    axes[0, 1].grid(True)
    
    axes[1, 0].plot(t, extracted, 'g-', label='Extracted')
    axes[1, 0].plot(t, mid_freq, 'r--', alpha=0.5, label=f'Original {f_mid} Hz')
    axes[1, 0].set_title('Extracted Band')
    axes[1, 0].set_xlabel('Time (s)')
    axes[1, 0].set_ylabel('Amplitude')
    axes[1, 0].set_xlim(0, 0.5)
    axes[1, 0].legend()
    axes[1, 0].grid(True)
    
    axes[1, 1].plot(frequencies, np.abs(fft_bandpass) / n_samples * 2, 'g-')
    axes[1, 1].set_title('Spectrum After Band-pass')
    axes[1, 1].set_xlabel('Frequency (Hz)')
    axes[1, 1].set_ylabel('Magnitude')
    axes[1, 1].set_xlim(0, 150)
    axes[1, 1].grid(True)
    
    plt.tight_layout()
    plt.show()

if WIDGETS_AVAILABLE:
    interact(
        plot_bandpass_filter,
        f_low=IntSlider(min=1, max=10, value=3, description='Low Freq'),
        f_mid=IntSlider(min=15, max=50, value=25, description='Mid Freq'),
        f_high=IntSlider(min=60, max=150, value=100, description='High Freq'),
        low_cutoff=IntSlider(min=5, max=40, value=15, description='Low Cut'),
        high_cutoff=IntSlider(min=30, max=80, value=40, description='High Cut')
    )
else:
    plot_bandpass_filter()

interactive(children=(IntSlider(value=3, description='Low Freq', max=10, min=1), IntSlider(value=25, descripti…

## 3. Interactive Notch Filter

Remove a specific interference frequency (e.g., 60 Hz power line noise).

In [4]:
def plot_notch_filter(signal_freq=10, interference_freq=60, notch_width=2):
    """Demonstrate notch filtering."""
    sample_rate = 1000
    duration = 2.0
    t = np.linspace(0, duration, int(sample_rate * duration), endpoint=False)
    
    # Create signal with interference
    desired = np.sin(2 * np.pi * signal_freq * t)
    interference = 0.8 * np.sin(2 * np.pi * interference_freq * t)
    contaminated = desired + interference
    
    # FFT and notch filter
    n_samples = len(contaminated)
    fft_result = np.fft.rfft(contaminated)
    frequencies = np.fft.rfftfreq(n_samples, d=1/sample_rate)
    
    notch_mask = (frequencies >= interference_freq - notch_width) & \
                 (frequencies <= interference_freq + notch_width)
    fft_notched = fft_result.copy()
    fft_notched[notch_mask] = 0
    cleaned = np.fft.irfft(fft_notched)
    
    # Plot
    fig, axes = plt.subplots(2, 2, figsize=(12, 8))
    
    axes[0, 0].plot(t, contaminated, 'b-')
    axes[0, 0].set_title(f'Contaminated: {signal_freq} Hz + {interference_freq} Hz interference')
    axes[0, 0].set_ylabel('Amplitude')
    axes[0, 0].set_xlim(0, 0.5)
    axes[0, 0].grid(True)
    
    axes[0, 1].plot(frequencies, np.abs(fft_result) / n_samples * 2, 'b-')
    axes[0, 1].axvspan(interference_freq - notch_width, interference_freq + notch_width, 
                       alpha=0.3, color='red', label=f'Notch at {interference_freq} Hz')
    axes[0, 1].set_title('Spectrum (Before Notch)')
    axes[0, 1].set_ylabel('Magnitude')
    axes[0, 1].set_xlim(0, 100)
    axes[0, 1].legend()
    axes[0, 1].grid(True)
    
    axes[1, 0].plot(t, cleaned, 'g-', label='Cleaned')
    axes[1, 0].plot(t, desired, 'r--', alpha=0.5, label='Original')
    axes[1, 0].set_title('Signal After Notch Filter')
    axes[1, 0].set_xlabel('Time (s)')
    axes[1, 0].set_ylabel('Amplitude')
    axes[1, 0].set_xlim(0, 0.5)
    axes[1, 0].legend()
    axes[1, 0].grid(True)
    
    axes[1, 1].plot(frequencies, np.abs(fft_notched) / n_samples * 2, 'g-')
    axes[1, 1].set_title('Spectrum After Notch')
    axes[1, 1].set_xlabel('Frequency (Hz)')
    axes[1, 1].set_ylabel('Magnitude')
    axes[1, 1].set_xlim(0, 100)
    axes[1, 1].grid(True)
    
    plt.tight_layout()
    plt.show()

if WIDGETS_AVAILABLE:
    interact(
        plot_notch_filter,
        signal_freq=IntSlider(min=5, max=30, value=10, description='Signal (Hz)'),
        interference_freq=IntSlider(min=40, max=100, value=60, description='Interference'),
        notch_width=IntSlider(min=1, max=10, value=2, description='Notch Width')
    )
else:
    plot_notch_filter()

interactive(children=(IntSlider(value=10, description='Signal (Hz)', max=30, min=5), IntSlider(value=60, descr…

## 4. Window Functions and Spectral Leakage

When a signal doesn't contain an exact number of cycles, FFT produces "spectral leakage". Window functions reduce this effect.

In [5]:
def plot_windowing(frequency=7.3, window_type='none'):
    """Explore windowing and spectral leakage."""
    sample_rate = 100
    duration = 1.0
    
    t = np.linspace(0, duration, int(sample_rate * duration), endpoint=False)
    signal = np.sin(2 * np.pi * frequency * t)
    n_samples = len(signal)
    
    # Apply window
    windows = {
        'none': np.ones(n_samples),
        'hann': np.hanning(n_samples),
        'hamming': np.hamming(n_samples),
        'blackman': np.blackman(n_samples)
    }
    window = windows[window_type]
    windowed_signal = signal * window
    
    # FFT
    fft_result = np.fft.rfft(windowed_signal)
    frequencies = np.fft.rfftfreq(n_samples, d=1/sample_rate)
    magnitude_db = 20 * np.log10(np.abs(fft_result) / n_samples + 1e-10)
    
    fig, axes = plt.subplots(1, 3, figsize=(14, 4))
    
    axes[0].plot(t, signal, 'b-', alpha=0.5, label='Original')
    axes[0].plot(t, windowed_signal, 'g-', label='Windowed')
    axes[0].set_xlabel('Time (s)')
    axes[0].set_ylabel('Amplitude')
    axes[0].set_title(f'Signal ({frequency} Hz) with {window_type} window')
    axes[0].legend()
    axes[0].grid(True)
    
    axes[1].plot(window, 'purple')
    axes[1].set_xlabel('Sample')
    axes[1].set_ylabel('Weight')
    axes[1].set_title(f'{window_type.capitalize()} Window Shape')
    axes[1].grid(True)
    
    axes[2].plot(frequencies, magnitude_db, 'b-')
    axes[2].axvline(x=frequency, color='r', linestyle='--', alpha=0.5, 
                    label=f'True: {frequency} Hz')
    axes[2].set_xlabel('Frequency (Hz)')
    axes[2].set_ylabel('Magnitude (dB)')
    axes[2].set_title('Spectrum')
    axes[2].set_xlim(0, 20)
    axes[2].set_ylim(-80, 0)
    axes[2].legend()
    axes[2].grid(True)
    
    plt.tight_layout()
    plt.show()

if WIDGETS_AVAILABLE:
    interact(
        plot_windowing,
        frequency=FloatSlider(min=5, max=15, step=0.1, value=7.3, description='Frequency (Hz)'),
        window_type=Dropdown(options=['none', 'hann', 'hamming', 'blackman'], 
                            value='none', description='Window')
    )
else:
    plot_windowing()

interactive(children=(FloatSlider(value=7.3, description='Frequency (Hz)', max=15.0, min=5.0), Dropdown(descri…

## 5. Compare All Window Functions

In [6]:
def compare_windows(frequency=7.3):
    """Compare spectral leakage across different windows."""
    sample_rate = 100
    duration = 1.0
    
    t = np.linspace(0, duration, int(sample_rate * duration), endpoint=False)
    signal = np.sin(2 * np.pi * frequency * t)
    n_samples = len(signal)
    frequencies = np.fft.rfftfreq(n_samples, d=1/sample_rate)
    
    windows = {
        'No Window': np.ones(n_samples),
        'Hann': np.hanning(n_samples),
        'Hamming': np.hamming(n_samples),
        'Blackman': np.blackman(n_samples)
    }
    
    fig, axes = plt.subplots(2, 2, figsize=(12, 8))
    
    for ax, (name, window) in zip(axes.flat, windows.items()):
        windowed = signal * window
        fft_result = np.fft.rfft(windowed)
        magnitude_db = 20 * np.log10(np.abs(fft_result) / n_samples + 1e-10)
        
        ax.plot(frequencies, magnitude_db, 'b-')
        ax.axvline(x=frequency, color='r', linestyle='--', alpha=0.5)
        ax.set_xlabel('Frequency (Hz)')
        ax.set_ylabel('Magnitude (dB)')
        ax.set_title(name)
        ax.set_xlim(0, 20)
        ax.set_ylim(-80, 0)
        ax.grid(True)
    
    plt.suptitle(f'Spectral Leakage Comparison ({frequency} Hz signal)', fontsize=14)
    plt.tight_layout()
    plt.show()

if WIDGETS_AVAILABLE:
    interact(
        compare_windows,
        frequency=FloatSlider(min=5, max=15, step=0.1, value=7.3, description='Frequency (Hz)')
    )
else:
    compare_windows()

interactive(children=(FloatSlider(value=7.3, description='Frequency (Hz)', max=15.0, min=5.0), Output()), _dom…

## 6. Spectral Analysis of Musical Chord

In [7]:
def plot_chord_spectrum(note1='C4', note2='E4', note3='G4', add_harmonics=False):
    """Analyze the spectrum of a musical chord."""
    # Note frequencies
    notes = {
        'C4': 261.63, 'D4': 293.66, 'E4': 329.63, 'F4': 349.23,
        'G4': 392.00, 'A4': 440.00, 'B4': 493.88, 'C5': 523.25
    }
    
    f1, f2, f3 = notes[note1], notes[note2], notes[note3]
    
    sample_rate = 8000
    duration = 0.5
    t = np.linspace(0, duration, int(sample_rate * duration), endpoint=False)
    
    # Generate chord
    signal = (0.5 * np.sin(2 * np.pi * f1 * t) +
              0.4 * np.sin(2 * np.pi * f2 * t) +
              0.4 * np.sin(2 * np.pi * f3 * t))
    
    if add_harmonics:
        signal += (0.2 * np.sin(2 * np.pi * 2 * f1 * t) +
                   0.15 * np.sin(2 * np.pi * 2 * f2 * t) +
                   0.15 * np.sin(2 * np.pi * 2 * f3 * t))
    
    # FFT
    n_samples = len(signal)
    fft_result = np.fft.rfft(signal)
    frequencies = np.fft.rfftfreq(n_samples, d=1/sample_rate)
    magnitude = np.abs(fft_result) / n_samples * 2
    
    fig, axes = plt.subplots(1, 2, figsize=(12, 4))
    
    axes[0].plot(t[:400], signal[:400], 'b-')
    axes[0].set_xlabel('Time (s)')
    axes[0].set_ylabel('Amplitude')
    axes[0].set_title(f'Chord: {note1} + {note2} + {note3}')
    axes[0].grid(True)
    
    axes[1].plot(frequencies, magnitude, 'b-')
    axes[1].axvline(x=f1, color='r', linestyle='--', alpha=0.5, label=f'{note1}: {f1:.0f} Hz')
    axes[1].axvline(x=f2, color='g', linestyle='--', alpha=0.5, label=f'{note2}: {f2:.0f} Hz')
    axes[1].axvline(x=f3, color='orange', linestyle='--', alpha=0.5, label=f'{note3}: {f3:.0f} Hz')
    axes[1].set_xlabel('Frequency (Hz)')
    axes[1].set_ylabel('Magnitude')
    axes[1].set_title('Frequency Spectrum')
    axes[1].set_xlim(0, 1200 if add_harmonics else 600)
    axes[1].legend()
    axes[1].grid(True)
    
    plt.tight_layout()
    plt.show()

if WIDGETS_AVAILABLE:
    note_options = ['C4', 'D4', 'E4', 'F4', 'G4', 'A4', 'B4', 'C5']
    interact(
        plot_chord_spectrum,
        note1=Dropdown(options=note_options, value='C4', description='Note 1'),
        note2=Dropdown(options=note_options, value='E4', description='Note 2'),
        note3=Dropdown(options=note_options, value='G4', description='Note 3'),
        add_harmonics=Checkbox(value=False, description='Add Harmonics')
    )
else:
    plot_chord_spectrum()

interactive(children=(Dropdown(description='Note 1', options=('C4', 'D4', 'E4', 'F4', 'G4', 'A4', 'B4', 'C5'),…

## 7. Real-Time Signal Simulation

Combine multiple effects: signal + interference + noise, then filter.

In [8]:
def full_signal_processing(signal_freq=10, interference_freq=60, 
                           noise_level=0.3, cutoff=40, use_window=False):
    """Complete signal processing pipeline."""
    sample_rate = 1000
    duration = 2.0
    t = np.linspace(0, duration, int(sample_rate * duration), endpoint=False)
    n_samples = len(t)
    
    # Create signals
    clean = np.sin(2 * np.pi * signal_freq * t)
    interference = 0.5 * np.sin(2 * np.pi * interference_freq * t)
    np.random.seed(42)
    noise = noise_level * np.random.randn(n_samples)
    
    contaminated = clean + interference + noise
    
    # Apply window if requested
    if use_window:
        window = np.hanning(n_samples)
        process_signal = contaminated * window
    else:
        process_signal = contaminated
    
    # FFT
    fft_result = np.fft.rfft(process_signal)
    frequencies = np.fft.rfftfreq(n_samples, d=1/sample_rate)
    
    # Low-pass filter
    fft_filtered = fft_result.copy()
    fft_filtered[frequencies > cutoff] = 0
    filtered = np.fft.irfft(fft_filtered)
    
    # Plot
    fig, axes = plt.subplots(3, 2, figsize=(14, 10))
    
    axes[0, 0].plot(t, clean, 'g-', alpha=0.7)
    axes[0, 0].set_title(f'Clean Signal ({signal_freq} Hz)')
    axes[0, 0].set_ylabel('Amplitude')
    axes[0, 0].grid(True)
    
    axes[0, 1].plot(frequencies, np.abs(np.fft.rfft(clean)) / n_samples * 2, 'g-')
    axes[0, 1].set_title('Clean Spectrum')
    axes[0, 1].set_xlim(0, 100)
    axes[0, 1].grid(True)
    
    axes[1, 0].plot(t, contaminated, 'b-', alpha=0.7)
    axes[1, 0].set_title(f'Contaminated (+ {interference_freq} Hz + noise)')
    axes[1, 0].set_ylabel('Amplitude')
    axes[1, 0].grid(True)
    
    axes[1, 1].plot(frequencies, np.abs(fft_result) / n_samples * 2, 'b-')
    axes[1, 1].axvline(x=cutoff, color='r', linestyle='--', label=f'Cutoff: {cutoff} Hz')
    axes[1, 1].set_title('Contaminated Spectrum')
    axes[1, 1].set_xlim(0, 100)
    axes[1, 1].legend()
    axes[1, 1].grid(True)
    
    axes[2, 0].plot(t, filtered, 'r-', label='Filtered')
    axes[2, 0].plot(t, clean, 'g--', alpha=0.5, label='Original')
    axes[2, 0].set_title('Filtered Signal')
    axes[2, 0].set_xlabel('Time (s)')
    axes[2, 0].set_ylabel('Amplitude')
    axes[2, 0].legend()
    axes[2, 0].grid(True)
    
    axes[2, 1].plot(frequencies, np.abs(fft_filtered) / n_samples * 2, 'r-')
    axes[2, 1].set_title('Filtered Spectrum')
    axes[2, 1].set_xlabel('Frequency (Hz)')
    axes[2, 1].set_xlim(0, 100)
    axes[2, 1].grid(True)
    
    plt.tight_layout()
    plt.show()
    
    error = np.sqrt(np.mean((clean - filtered)**2))
    print(f"RMS error vs original: {error:.4f}")

if WIDGETS_AVAILABLE:
    interact(
        full_signal_processing,
        signal_freq=IntSlider(min=5, max=30, value=10, description='Signal (Hz)'),
        interference_freq=IntSlider(min=40, max=100, value=60, description='Interference'),
        noise_level=FloatSlider(min=0, max=1.0, step=0.1, value=0.3, description='Noise'),
        cutoff=IntSlider(min=15, max=80, value=40, description='Cutoff (Hz)'),
        use_window=Checkbox(value=False, description='Use Hann Window')
    )
else:
    full_signal_processing()

interactive(children=(IntSlider(value=10, description='Signal (Hz)', max=30, min=5), IntSlider(value=60, descr…

## Summary

In this notebook, we covered practical applications of FFT:

1. **Low-pass Filtering** - Removing high-frequency noise
2. **Band-pass Filtering** - Extracting specific frequency ranges
3. **Notch Filtering** - Removing specific interference frequencies
4. **Windowing** - Reducing spectral leakage
5. **Spectral Analysis** - Analyzing musical signals
6. **Complete Pipeline** - Combining multiple techniques

These techniques form the foundation for signal processing applications in audio, communications, scientific instrumentation, and many other fields.